In [16]:
# !pip install langchain langchain_community
# !pip install sentence-transformers
# !pip install pypdf
# !pip install tiktoken
# !pip install ragas
# !pip install bert-score
# !pip install datasets peft bitsandbytes

## Maestría en Inteligencia Artificial Aplicada (MNA)
### Proyecto Integrador
### Dra. Grettel Barceló Alonso / Dr. Carlos Alberto Villaseñor Padilla
### Avance 4. Implementación de Arquitectura RAG

### Integrantes
- A01794457 - Iossif Moises Palli Laura
- A01793984 - Brenda Zurazy Rodríguez Pérez
- A01794630 - Jesús Ramseths Echeverría Rivera

In [33]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from bert_score import score
from peft import LoraConfig, get_peft_model, PeftModel
from transformers import BitsAndBytesConfig
from datasets import Dataset
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, pipeline
import torch
import re
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from huggingface_hub import login
# Autenticar con Hugging Face utilizando un token personal
login(os.environ["HUGGING_KEY"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Carga de los documentos

In [7]:
# Cargar un documento PDF y extraer sus páginas
loader = PyPDFLoader('./Ley de Fondos de Inversión.pdf', extract_images=False)
pages = loader.load()
# Dividir el texto en fragmentos más pequeños
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size = 256)
docs = text_splitter.split_documents(pages)

In [8]:
# Limpiar el texto de los documentos
for doc in docs:
    # Se elimina espacios y saltos de línea al inicio y al final
    text = doc.page_content.strip()
    text = text.replace('\n', ' ')

    # Se wliminan espacios duplicados
    text = re.sub(r'[ \t]+', ' ', text)

    # Actualizar
    doc.page_content = text

# Filtrar documentos vacíos
docs = [doc for doc in docs if doc.page_content.strip() != '']

In [9]:
# Imprimir el contenido de la primera página procesada
print(docs[0].page_content)

Para uso interno Para uso interno (3) LEY DE FONDOS DE INVERSIÓN Publicada en el Diario Oficial de la Federación el 4 de junio de 2001. Actualizada con las modificaciones publicadas en el propio Diario el 28 de junio de 2007, 10 de enero y 13 de junio de 2014, 20 de mayo de 2021 , 28 de diciembre de 2023 y 24 de enero de 2024.


### Creación de Embeddings

In [10]:
# Cargar un modelo de embeddings de Hugging Face para transformar el texto
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")

# Definir la ruta donde se guardarán los embeddings
persist_path = "../embeddings_db_rag"
# Crear un almacén vectorial con los documentos y sus embeddings
vector_store = SKLearnVectorStore.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_path=persist_path,
    serializer="parquet"
)

<ipython-input-10-25dc8f5532f5>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-MiniLM-L6-cos-v1")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public m

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
# Fuerza a guardar los embeddings en la DB
vector_store.persist()

In [12]:
# Crear el "retriever", que es el mecanismo que buscará entre los embeddings
retriever = vector_store.as_retriever()

### Model Load

In [13]:
# Cargar un modelo de lenguaje preentrenado (Llama) para generación de texto
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(
    model_name,
    device_map='auto'
)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [18]:
# Cargar los adaptadores LoRa del Fine Tuning
lora_weights_path = "./llama-3.2-3B-Instruct-FN/"

model = PeftModel.from_pretrained(
    model,
    lora_weights_path,
    device_map="auto",
)

In [19]:
# Definir pipeline para la generación de texto
generate_text = pipeline(
    task='text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

In [20]:
# Configurar el modelo de lenguaje dentro del pipeline de Hugging Face
llm = HuggingFacePipeline(pipeline=generate_text)

<ipython-input-20-a8099b2d8111>:2: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generate_text)


In [21]:
prompt_template = """
Utiliza el siguiente contexto para responder la pregunta al final de manera muy corta en un solo reglón.

Contexto:
{context}

Pregunta: {question}

Respuesta:
"""

In [22]:
prompt = PromptTemplate(
    input_variables=["context","question"],
    template=prompt_template
)

In [23]:
# Crear la cadena de preguntas y respuestas (QA) basada en el sistema RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt,
    }
)

### Evaluación

In [24]:
from ragas import evaluate
from ragas.metrics import answer_relevancy, answer_similarity

In [27]:
# Carga de datos prueba
test_df = pd.read_csv('./test_q_a.csv')

In [28]:
# Se extrae la respuesta del contenido generado por el LLM
def extract_answer(result):
    output = result['result']

    if "Respuesta:" in output:
        answer = output.split("Respuesta:")[-1].strip()
    else:
        answer = output.strip()

    return answer

In [29]:
# Listas para guardar los respuestas predichas y los documentos recuperados
results = []
contexts = []
for question in test_df['question']:
    result = qa_chain({"query": question})
    answer = extract_answer(result)
    results.append(answer)
    sources = result["source_documents"]
    contents = []
    for i in range(len(sources)):
        contents.append(sources[i].page_content)
    contexts.append(contents)

<ipython-input-29-8e94fdc17431>:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
You seem to be using the pipelines sequentiall

In [30]:
# Se crea el diccionario con los elementos necesarios para evaluar
test_set = {
    "question": test_df['question'].tolist(),
    "answer": results,
    "contexts": contexts,
    "ground_truth": test_df['answer'].tolist()
}
dataset = Dataset.from_dict(test_set)
get_score = evaluate(dataset, metrics=[answer_relevancy, answer_similarity])
score_df = get_score.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `dict[str, any]` but got `EvaluationResult` with value `{'answer_relevancy': 0.87...tic_similarity': 0.8643}` - serialized value may not be as expected
  Expected `dict[str, any]` but got `EvaluationResult` with value `{'answer_relevancy': 0.87...tic_similarity': 0.8643}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [31]:
score_df[['answer_relevancy', 'semantic_similarity']].mean(axis=0)

,0
answer_relevancy,0.875857
semantic_similarity,0.864348


In [34]:
def calculate_bert_score_avg(generated_responses, real_responses, model_name):

  # Se obtiene la precisión, recall y F1 de acuerdo a la comparación.
  P, R, F1 = score(generated_responses, real_responses, lang='es', verbose=True)

  return F1.mean().item(), P.mean().item(), R.mean().item()

In [36]:
f1_score, precision, recall = calculate_bert_score_avg(results, test_df['answer'].tolist(), "GraphRAG")
print(f"Puntaje F1 promedio: {f1_score:.4f}")
print(f"Puntaje de Precisión promedio: {precision:.4f}")
print(f"Puntaje de Recall promedio: {recall:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.12 seconds, 172.92 sentences/sec
Puntaje F1 promedio: 0.7427
Puntaje de Precisión promedio: 0.7305
Puntaje de Recall promedio: 0.7571


### Conclusión

El uso de Retrieval-Augmented Generation (RAG) proporciona una poderosa combinación de recuperación de información y generación de texto. En este caso, el sistema permite recuperar información relevante de un documento de manera efectiva y generar respuestas precisas y coherentes.

El modelo RAG ofrece una ventaja clave al integrar un retriever que busca el contexto adecuado en una base de datos vectorial de documentos y luego utiliza un modelo generador para crear respuestas basadas en ese contexto.

Esto es especialmente útil en situaciones donde la información se distribuye en múltiples fuentes o documentos y se necesita una respuesta concisa.